In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from laspy.file import File
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

import sys

sys.path.insert(0,'..')
from point_density_functions import *
%load_ext autoreload
%matplotlib notebook

In [ ]:
file_dir = '../../Data/parking_lot/'

filenames =list(pd.read_csv(file_dir+"filenames.txt",header=None)[0])
pt_files = list(pd.read_csv(file_dir+"pt_files.txt",header=None)[0])

nyc_file_dir = '../../Data/NYC_topo/'
nyc_pt_file = ['las_points_NYC_flightid_975172.lz']

# Corresponds to LAS 1.2 Point Data Record Format 1
columns_dublin_pt_cloud = [
    'X',
    'Y',
    'Z',
    'intensity',
    'return_number_byte',
    'classification_byte',
    'scan_angle',
    'user_data',
    'pt_src_id',
    'gps_time']

columns_point_cloud = [
    'X','Y','Z',
    'intensity',
    'flag_byte',
    'classification_flags',
    'classification_byte',
    'user_data',
    'scan_angle',
    'pt_src_id',
    'gps_time']

In [ ]:
# Extract points within a square around the desired point

# Parking Lot
# pt_x = 977037.343
# pt_y = 174586.034

# Corner of building
# pt_x_bldg = 977229.375
# pt_y_bldg = 174579.42

# Top of building
# pt_x = 977229.375
# pt_y = 174579.42

# Projects in back parking lot
# pt_x = 977458.238
# pt_y = 173302.388

# Solar panel
# pt_x = 977682.975
# pt_y = 174148.192

In [ ]:
# This works
# for filename in filenames:
#     create_df_hd5(file_dir,filename,columns_point_cloud)

## Flat surface
Identifying points in a parking lot to assess how consistently flat they are.  
Center point: 40.645789, -74.025951  
Easting - 977048.434  
Northing - 174555.792

### Point density - waterside parking lot

In [ ]:
# Big parking lot rectangle
def rectangle(pt1,pt2,y_length,x_length):
    '''
    Function returns uv_inv and w, for use in selecting points within the rectangle
    Note: This function only works in 2D (horizontal plane)
    Inputs:
    pt1 - 2x1 numpy array with x and y coordinate for bottom point
    pt2 - 2xy numpy array with x and y coordinate for top point
    y_length - bottom-to-top length (positive is in direction of top from bottom point)
    x_length - left-to-right length (positive means pts are on left border, negative means they're on right)
    Outputs:
    uv_inv: 2xy numpy array - u and v are the sides of the rectangle.  uv = [u v] is a matrix with u and v as columns.
    w: 2x1 numpy array with (x,y) coordinates of reference (bottom) point.
    
    Reference: https://math.stackexchange.com/questions/190111/how-to-check-if-a-point-is-inside-a-rectangle
    '''
    unit_u = (pt2 - pt1)/np.linalg.norm(pt2-pt1)
    unit_v = np.array([unit_u[1],-1*unit_u[0]])
    u = unit_u*y_length
    v = unit_v*x_length
    uv = np.array([u,v]).T
    uv_inv = np.linalg.inv(uv)
    w = pt1
    return uv_inv,w

pt1 = np.array([976600.43, 173886.11,0])
pt2 = np.array([976774.81, 174083.03,0])
pt3 = np.array([976606.419, 173880.806,0]) # Backed into from previous 2D v calculation
uv_inv,w = rectangle(pt1[:2],pt2[:2],800,-80)


#rectangle_points_laefer = grab_points_big_rect(pt_files,file_dir,uv_inv,w)
#rectangle_points_nyc = grab_points_big_rect(nyc_pt_file,nyc_file_dir,uv_inv,w)

In [ ]:
# Load the pre-pickled DF's
rectangle_points_laefer = pd.read_pickle(file_dir+"rectangle_points_laefer.pkl")
rectangle_points_nyc = pd.read_pickle(file_dir+"rectangle_points_nyc.pkl")

In [ ]:
# Remove outliers and print the count: z < 0, z > 30
def remove_vertical_outliers(rectangle_points,z_low,z_high):
    outliers = rectangle_points[(rectangle_points['z_scaled']<z_low) | (rectangle_points['z_scaled']>z_high)].index
    rectangle_points = rectangle_points.drop(outliers)
    print("Number of outliers: {}".format(len(outliers)))
    return rectangle_points
rectangle_points_laefer = remove_vertical_outliers(rectangle_points_laefer,0,30)
rectangle_points_nyc = remove_vertical_outliers(rectangle_points_nyc,0,30)

## Sampling squares and Statistics

In [ ]:
def center_point_sample(num_points,
                        bottom_left_pt,top_left_pt,bottom_right_pt=None,
                        u_length=800,v_length=-80,
                        border=[0.05,0.05]):
    '''
    Inputs:
    bottom_left_pt - 3x1 numpy array with xyz coordinate 
    top_left_pt - 3x1 numpy array with xyz coordinate 
    bottom_right_pt - 3x1 numpy array with xyz coordinate
    u_length - length in the u direction (bottom_left_pt -> top_left_pt)
    v_length - length in the u direction (bottom_left_pt -> bottom_right_pt)
    border - portion of unit square on each edge to avoid
    
    Output:
    num_points x 2 numpy array of (x,y) values
    '''
    n_dim = len(bottom_left_pt)
    
    # 2-D case (xy plane)
    if n_dim == 2:
    # Calculate uv and w for use in translation
        unit_u = (top_left_pt - bottom_left_pt)/np.linalg.norm(top_left_pt-bottom_left_pt)
        unit_v = np.array([unit_u[1],-1*unit_u[0]])
        u = unit_u*u_length
        v = unit_v*v_length
        print("V: ",v)
        uv = np.array([u,v]).T
        
    if n_dim == 3:
        unit_u = (top_left_pt - bottom_left_pt)/np.linalg.norm(top_left_pt-bottom_left_pt)
        unit_v = (bottom_right_pt - bottom_left_pt)/np.linalg.norm(bottom_right_pt - bottom_left_pt)
        u = unit_u*u_length
        v = unit_v*v_length
        uv = np.array([u,v]).T # Why the transpose?
            
    w = bottom_left_pt
    # Select random point on unit square within border
    border = np.array(border)
    square_side = np.array([1 - (2*border[0]),1 - (2*border[1])])
    st = border.reshape(2,1) + square_side.reshape(2,1)*np.random.rand(2,num_points)
#     if n_dim ==3:
#         st = np.concatenate((st,np.zeros(num_points).reshape(1,num_points)),axis=1)
    return (uv @ st + w.reshape((n_dim,1))).T

def in_square(rectangle_points,center_point,feet_from_point):
    '''
    Filters dataframe for points less than feet_from_point from center_point
    Inputs:
    rectangle_points - (n x 3+) dataframe with fields x_scaled, y_scaled, and z_scaled
    center_point - 2x1 (x,y) point within rectangle
    feet_from_point - scalar
    
    Output:
    square_points - (n x 3+) filtered dataframe
    '''
    square_points = rectangle_points[ (rectangle_points['x_scaled'] < center_point[0] + feet_from_point)
            &(rectangle_points['x_scaled'] > center_point[0] - feet_from_point) 
            &(rectangle_points['y_scaled'] < center_point[1] + feet_from_point)
            &(rectangle_points['y_scaled'] > center_point[1] - feet_from_point)
          ]
    return square_points

# feet_from_point = 3
# center_points = center_point_sample(100,pt1,pt2,pt3,u_length=800,v_length=-80,border=0.05)
# center_points = center_point_sample(100,pt1[:2],pt2[:2],u_length=800,v_length=-80,border=0.05)

In [ ]:
class FlightPath(object):
    '''
    FlightPath is an object that stores data about a single flight path for a given sample square.
    
    Attributes:
    flight_id - integer identifier of the flight (typically 0-40)
    norm_vector - 3x1 numpy array of the xyz coordinates of the norm vector
    avg_dist_from_plane - scalar average distance from fitted plane (fitted over all flight paths)
        for all points in the flight path
    
    '''
    def __init__(self,flight_id,norm_vector,avg_dist_from_plane=None):
        self.flight_id = flight_id
        self.norm_vector = norm_vector
        self.avg_dist_from_plane = avg_dist_from_plane
    
    def sd_dist_from_plane(self,square_points):
        self.sd_dist = np.std(square_points['dist_from_plane'])

class SampleSquare(object):
    '''
    Object to store all statistics for one sample square.
    
    Attributes:
    x,y - scalar coordinates of center point defining the square
    feet_from_point - scalar 1/2 length of one side of square
    nyc_/laefer_ flight_list - list of FlightPath objects from nyc or laefer dataset
    delta_h_matrix - k x k numpy array, where k is the number of flight paths and the entries are the difference
        in avg. point distance from the plane fit to the total point cloud 
    cosine_sim_matrix = k x k numpy array, where k is the number of flight paths and the entries are 
    cosine similarities between their normal vectors.
    '''
    def __init__(self, flight_list_laefer, flight_list_nyc ,x=None, y=None, feet_from_point=None):
        self.x = x
        self.y = y
        self.feet_from_point = feet_from_point
        self.flight_list_laefer = flight_list_laefer
        self.flight_list_nyc = flight_list_nyc
        self.delta_h_matrix_nyc = self.delta_h_internal(self.flight_list_nyc)
        self.delta_h_matrix_laefer = self.delta_h_internal(self.flight_list_laefer)
        self.cosine_sim_matrix_nyc = self.cosine_sim_internal(self.flight_list_nyc)
        self.cosine_sim_matrix_laefer = self.cosine_sim_internal(self.flight_list_laefer)

        self.phi_nyc_total = self.phi_internal(self.flight_list_nyc,sample=False)
        self.phi_laefer_total = self.phi_internal(self.flight_list_laefer,sample=False)
        self.phi_nyc_sample = self.phi_internal(self.flight_list_nyc,sample=True)
        self.phi_laefer_sample = self.phi_internal(self.flight_list_laefer,sample=True)

        
    def delta_h_internal(self,flight_list):
        # Calculates difference in avg dist from plane for all flight path pairs, returns a matrix
        delta_h_matrix = np.zeros((len(flight_list)-2,len(flight_list)-2))
        for i,flight_i in enumerate(flight_list[2:]): # Skip total and sampled
            for j,flight_j in enumerate(flight_list[2:]): # Skip total and sampled
                delta_h_matrix[i,j] = abs(flight_i.avg_dist_from_plane - flight_j.avg_dist_from_plane)
        return delta_h_matrix

    def cosine_sim_internal(self,flight_list):
        # Calculates cosine similarity for normal vectors of all flight path pairs, returns a matrix
        cosine_sim_matrix = np.zeros((len(flight_list)-2,len(flight_list)-2))
        for i,flight_i in enumerate(flight_list[2:]): # Skip total and sampled
            for j,flight_j in enumerate(flight_list[2:]): # Skip total and sampled
                delta_h_matrix[i,j] = flight_i.norm_vector @ flight_j.norm_vector
        return delta_h_matrix
    
    def phi_internal(self,flight_list,sample=False):
        avg_flight_paths = np.mean([flight.sd_dist for flight in flight_list[2:]])
        if sample:
            phi = flight_list[1].sd_dist / avg_flight_paths
        else:
            phi = flight_list[0].sd_dist / avg_flight_paths
        return phi

In [ ]:
def create_flight_list(square_points):
    '''
    flight_list creates a list of FlightPath objects, 
    1 for each unique flight_id plus 2 more (total and total_sampled).
    FlightPath object contains flight_id, norm_vector, std deviation of point distance from fitted plane.
    flight_id = -100: Full dataset
    flight_id = -200: Full dataset, sampled to the avg number of points in a single flight path
    
    Input:
    square_points: Dataframe for the square-around-a-point, including x_scaled,y_scaled, and z_scaled
    
    Output:
    flight_list (described above)
    '''
    
    # fit a plane for each flight
    flight_list = []

    # Full dataset
    norm_vector,_,square_points_total,_ = plane_fit(square_points)
    flightpath = FlightPath(-100,norm_vector)
    flightpath.sd_dist_from_plane(square_points_total)
    flight_list.append(flightpath)

    # Full dataset, sampled down
    flight_count = len(square_points['flight_id'].unique())
    density = square_points.shape[0] / flight_count
    square_points_sampled = square_points.sample(n=int(density))

    norm_vector,_,square_points_sample,_ = plane_fit(square_points_sampled)
    flightpath = FlightPath(-200,norm_vector)
    flightpath.sd_dist_from_plane(square_points_sampled)
    flight_list.append(flightpath)


    for flight_id in square_points['flight_id'].unique():
        # Avg distance from total point cloud plane - filtering square_points_total
        avg_dist_from_plane = square_points_total[square_points_total['flight_id']==flight_id]['dist_from_plane'].mean()

        norm_vector,_,square_points_flight,_ = plane_fit( \
                                               square_points[square_points['flight_id']==flight_id])
        flightpath = FlightPath(flight_id,norm_vector,avg_dist_from_plane)
        flightpath.sd_dist_from_plane(square_points_flight)
        flight_list.append(flightpath)       
        
    return flight_list

In [ ]:
# Aggregators
pt_density_nyc_list, pt_density_laefer_list = [],[]
avg_height_diff = []
sd_height_nyc, sd_height_laefer = [],[]
pts_thrown_out = 0
SampleFlightList = []
for center_point in center_points:
    square_points_nyc = in_square(rectangle_points_nyc,center_point[:2],feet_from_point)
    square_points_laefer = in_square(rectangle_points_laefer,center_point[:2],feet_from_point)
    
    laefer_flight_count = len(square_points_laefer['flight_id'].unique())
    nyc_flight_count = len(square_points_nyc['flight_id'].unique())
    laefer_density = square_points_laefer.shape[0] / laefer_flight_count
    nyc_density = square_points_nyc.shape[0]/nyc_flight_count
                            
#     print("Laefer: \n"+"*"*30)
#     print("Total points: {}".format(square_points_laefer.shape[0]))
#     print("Number of flights: {}".format(laefer_flight_count))
#     print("Points per flight: {}".format(laefer_density))
#     print("NYC: \n"+"*"*30)
#     print("Total points (NYC): {}".format(square_points_nyc.shape[0]))
#     print("Number of flights (NYC): {}".format(nyc_flight_count))
#     print("Points per flight: {}".format(nyc_density))    

    # If z_max > 10, <6 for either dataset, skip it
    if (square_points_nyc['z_scaled'].max()<10) & (square_points_nyc['z_scaled'].min()>6) & \
       (square_points_laefer['z_scaled'].max()<10) & (square_points_laefer['z_scaled'].min()>6):
        # Statistics!
                
        # Point density
        num_points_nyc = square_points_nyc.shape[0]
        pt_density_nyc_list.append(num_points_nyc / (4 * feet_from_point**2))        
        num_points_laefer = square_points_laefer.shape[0]
        pt_density_laefer_list.append(num_points_laefer / (4 * feet_from_point**2))        

        # Fit a plane
        norm_vector_nyc,_,square_points_nyc,_ = plane_fit(square_points_nyc)
        norm_vector_laefer,_,square_points_laefer,_ = plane_fit(square_points_laefer)

        # Flight path specifics
        laefer_flight_list = create_flight_list(square_points_laefer)
        nyc_flight_list = create_flight_list(square_points_nyc)
        ss = SampleSquare(laefer_flight_list, nyc_flight_list, x = center_point[0], \
                          y=center_point[1],feet_from_point=feet_from_point)
        SampleFlightList.append(ss)
        
    else:
        pts_thrown_out +=1

In [ ]:
sd_laefer_total = np.mean([ss.flight_list_laefer[0].sd_dist for ss in SampleFlightList])
sd_laefer_sample = np.mean([ss.flight_list_laefer[1].sd_dist for ss in SampleFlightList])

phis_laefer_total = [ss.phi_laefer_total for ss in SampleFlightList]
phis_laefer_sample = [ss.phi_laefer_sample for ss in SampleFlightList]

print("Laefer: \n"+"*"*30)
print("SD total: {:2.4f}".format(sd_laefer_total))
print("SD sample: {:2.4f}".format(sd_laefer_sample))
print("mean of phi_total: {:2.4f}".format(np.mean(phis_laefer_total)))
print("SD of phi_total:   {:2.4f}".format(np.std(phis_laefer_total)))
print("mean of phi_sample: {:2.4f}".format(np.mean(phis_laefer_sample)))
print("SD of phi_sample:   {:2.4f}".format(np.std(phis_laefer_sample)))

phis_nyc_total = [ss.phi_nyc_total for ss in SampleFlightList]
phis_nyc_sample = [ss.phi_nyc_sample for ss in SampleFlightList]

sd_nyc_total = np.mean([ss.flight_list_nyc[0].sd_dist for ss in SampleFlightList])
sd_nyc_sample = np.mean([ss.flight_list_nyc[1].sd_dist for ss in SampleFlightList])

print("\nNYC: \n"+"*"*30)
print("SD total: {:2.4f}".format(sd_nyc_total))
print("SD sample: {:2.4f}".format(sd_nyc_sample))
print("mean of phi_total: {:2.4f}".format(np.mean(phis_nyc_total)))
print("SD of phi_total:   {:2.4f}".format(np.std(phis_nyc_total)))
print("mean of phi_sample: {:2.4f}".format(np.mean(phis_nyc_sample)))
print("SD of phi_sample:   {:2.4f}".format(np.std(phis_nyc_sample)))

In [ ]:
'''
Laefer: 
******************************
SD total: 0.0389
SD sample: 0.0382
mean of phi_total: 1.2279
SD of phi_total:   0.1140
mean of phi_sample: 1.2030
SD of phi_sample:   0.2304

NYC: 
******************************
SD total: 0.0602
SD sample: 0.0544
mean of phi_total: 1.3172
SD of phi_total:   0.2091
mean of phi_sample: 1.2021
SD of phi_sample:   0.3375
'''

In [ ]:
print("NYC avg density: {:2.4f} (SD: {:2.4f})".format(np.mean(pt_density_nyc_list),np.std(pt_density_nyc_list)))
print("Laefer avg density: {:2.4f} (SD: {:2.4f})".format(np.mean(pt_density_laefer_list),np.std(pt_density_laefer_list)))
print("Points thrown out: {} ({:2.2%})".format(pts_thrown_out,pts_thrown_out/len(center_points)))

In [ ]:
# Pickle the points in the big rectangle
#rectangle_points_laefer.to_pickle("../../Data/parking_lot/rectangle_points_laefer.pkl")
rectangle_points_nyc.to_pickle("../../Data/parking_lot/rectangle_points_nyc.pkl")

In [ ]:
# Parking Lot
pt_x = 977037.343
pt_y = 174586.034
feet_from_pt = 5
# square_points_laefer = grab_points(pt_files,file_dir,pt_x,pt_y,feet_from_pt)
# Remove 3-point flight 200024
# square_points_laefer = square_points_laefer[square_points_laefer['flight_id']!='200024']
square_points_nyc = grab_points(nyc_pt_file,nyc_file_dir,pt_x,pt_y,feet_from_pt)

In [ ]:
# Fit a plane to each flight in laefer and nyc, and one overall plane
total_plane_norm,total_points = plane_fit(square_points_laefer)[:2]
nyc_plane_norm,nyc_points = plane_fit(square_points_nyc)[:2]
flight_ids = square_points_laefer['flight_id'].unique()
flight_plane_norms_dict ={}
for flight_id in flight_ids:
    flight_plane_norms_dict[flight_id] = plane_fit(square_points_laefer[square_points_laefer['flight_id']==flight_id])[:2]

In [ ]:
# Compare the normal vectors...cosine similarity?
# Dataframe of the cosine similarities
cosine_simarlity_df = pd.DataFrame(index = flight_ids,columns = flight_ids,dtype=float)
for flight_id_ix in flight_ids:
    for flight_id_col in flight_ids:
        cosine_simarlity_df.loc[flight_id_ix,flight_id_col] = float(flight_plane_norms_dict[flight_id_ix][0]@ \
                                                              flight_plane_norms_dict[flight_id_col][0])


In [ ]:
sns.heatmap(cosine_simarlity_df)

In [ ]:
# Calculate avg pt distance from flight plane and overall plane for each flight

square_points_laefer['dist_from_total_plane'] = np.array([np.dot(point,np.array([0,0,1])) for point in total_points])
square_points_nyc['dist_from_total_plane'] = np.array([np.dot(point,np.array([0,0,1])) for point in nyc_points])
# Compare whole process to splitting the points randomly instead of by flight

In [ ]:
print("Mean point distance from plane (laefer): {:2.4f}".format(abs(square_points_laefer['dist_from_total_plane']).mean()))
print("Mean point distance from plane (nyc): {:2.4f}".format(abs(square_points_nyc['dist_from_total_plane']).mean()))

## Vertical density
Identifying point at corner of building to quantify the vertical point density.  
Center point: 	40.645854, 	-74.025299  
Easting - 977229.375  
Northing - 174579.42

#### Laefer data

In [ ]:
%autoreload

# Outer wall
middle_pt = np.array([977223.71,174573.7,75.141])
right_pt = np.array([977221.44, 174571.037, 68.44])
left_pt = np.array([977226.47, 174577.23, 36.958])
bottom_left_pt = np.array([977225.752, 174576.37, 26.393])
top_left_pt = np.array([977225.752, 174576.37, 117.937])
bottom_right_pt = np.array([977221.605,174571.074,26.393])
feet_from_pt = 3.5

# Other side of the windows
# middle_pt = np.array([977214.86,174562.64,58.762])
# right_pt = np.array([977213.577,174561.01,111.614])
# left_pt = np.array([977217.294, 174565.64, 33.483])
# feet_from_pt = 3.5

# Extract from all files, the points within feet_from_pt in the xy-plane of the middle wall point
# square_points_vertical_laefer = grab_points(pt_files,file_dir,middle_pt[0],middle_pt[1],feet_from_pt)
# square_points_vertical_nyc = grab_points(nyc_pt_file,nyc_file_dir,middle_pt[0],middle_pt[1],feet_from_pt)

# Calculate norm_vector from 3 points, to define plane and extract the wall face
norm = np.cross(middle_pt - right_pt,(left_pt-right_pt))
norm = norm / np.linalg.norm(norm)

# Extract the wall face, above 15ft and below 120ft to avoid roof and grouund points
wall_face_laefer = grab_wall_face(square_points_vertical_laefer,norm, middle_pt,27,120,1)
wall_face_nyc = grab_wall_face(square_points_vertical_nyc,norm, middle_pt,27,120,1e-1)

# Fit a plane, create norm_vector, calculate dist_from_plane
norm_vector_laefer,_,wall_face_laefer,_ = plane_fit(wall_face_laefer) 
norm_vector_nyc,_,wall_face_nyc,_ = plane_fit(wall_face_nyc) 

In [ ]:
print("Pts in Laefer: {}".format(wall_face_laefer.shape[0]))
print("Pts in NYC: {}".format(wall_face_nyc.shape[0]))

In [ ]:
# Calculate the rectangle side length, based on points
u_length = np.linalg.norm(bottom_left_pt-top_left_pt)
v_length = np.linalg.norm(bottom_left_pt-bottom_right_pt)

# Sample points in wall
center_points = center_point_sample(100,
                    bottom_left_pt,top_left_pt,bottom_right_pt,
                    u_length=117.937-26.393,v_length=v_length,border=[0.08,0.28])

In [ ]:
for center_point in center_points:
    vert_square_points_nyc = in_vertical_square(wall_face_nyc,norm_vector_nyc,center_point,3.5/2,3.5)
    vert_square_points_laefer = in_vertical_square(wall_face_laefer,norm_vector_laefer,center_point,3.5/2,3.5)

In [ ]:
# Plotting
plotting_laefer,_ = prep_square_for_plotting(wall_face_nyc)
fig = px.scatter_3d(plotting_laefer, x='x_scaled', y='y_scaled', z='z_scaled',
              size = 'size_num',size_max = 12)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)"),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    )
fig.show()

In [ ]:
xy_norm = []
for index, pt in proj_on_plane.iterrows():
    norm = np.linalg.norm(pt[['x_scaled','y_scaled']] - center_points[25,:2])
    xy_norm.append(norm)
np.mean(xy_norm)

In [ ]:
vs_2.shape

In [ ]:
5.306*24.5

In [ ]:
'''
def in_square(rectangle_points,center_point,feet_from_point):
    '''
    Filters dataframe for points less than feet_from_point from center_point
    Inputs:
    rectangle_points - (n x 3+) dataframe with fields x_scaled, y_scaled, and z_scaled
    center_point - 2x1 (x,y) point within rectangle
    feet_from_point - scalar
    
    Output:
    square_points - (n x 3+) filtered dataframe
    '''
    square_points = rectangle_points[ (rectangle_points['x_scaled'] < center_point[0] + feet_from_point)
            &(rectangle_points['x_scaled'] > center_point[0] - feet_from_point) 
            &(rectangle_points['y_scaled'] < center_point[1] + feet_from_point)
            &(rectangle_points['y_scaled'] > center_point[1] - feet_from_point)
          ]
    return square_points
    '''

#### NYC data

In [ ]:
# Access the NYC header file
#inFile = File(nyc_file_dir+'975172.las', mode='r')

# Extract points within a square around the desired point
# Use pt_[]_bldg and feet_from_pt from previous cell

# Use same min's from previous so points line up.
square_points_nyc,_ = prep_square_for_plotting(square_points_nyc,min_list) 

#### 3D Plots

In [ ]:
fig = px.scatter_3d(square_points_bldg, x='x_plot', y='y_plot', z='z_plot',
              size='size_num',size_max = 12)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)"),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    )
fig.show()

In [ ]:
fig = px.scatter_3d(square_points_nyc, x='x_plot', y='y_plot', z='z_plot',
              size='size_num',size_max = 12)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)"),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    )
fig.show()

#### Extract the wall face, find normal vector, calculate vertical density

In [ ]:
pt_1 = np.array([37.7,10.04])
pt_2 = np.array([27.094,18.439])
wall_pt = (31.77,14.90,43.00)

# East? Facing Wall (farther from the little stump)
# pt_1 = np.array([20.59,15.02])
# pt_2 = np.array([10.34,2.93])
# wall_pt = (19.414,12.379,73.863)

wall_face = grab_wall_face(square_points_bldg,pt_1,pt_2,20,100,1e-2)
wall_face_nyc = grab_wall_face(square_points_nyc,pt_1,pt_2,20,100,5e-2)

In [ ]:
wall_face.shape

In [ ]:
wall_face_nyc.shape

In [ ]:
norm_vector,points,wall_face,pts_on_plane = plane_fit(wall_face)

In [ ]:
print(norm_vector)
print(norm_vector_nyc)

In [ ]:
norm_vector_nyc,points,wall_face_nyc,pts_on_plane_nyc = plane_fit(wall_face)

In [ ]:
fig = px.scatter_3d(wall_face, x='x_plot', y='y_plot', z='z_plot',
              color='flight_id',size='size_num',size_max = 12)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)"),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    )
fig.show()

In [ ]:
# Point in middle of wall - in xyz_plot coordinates (less the min of each coordinate)
# It's best to highlight a point in the plot above and use that.

feet_from_pt = 3

In [ ]:
# Calculate density for both datasets   
vertical_point_density(square_points_nyc,norm_vector_nyc,wall_pt,feet_from_pt)
vertical_point_density(square_points_bldg,norm_vector,wall_pt,feet_from_pt)

In [ ]:
fig = px.scatter_3d(vertical_square, x='x_plot', y='y_plot', z='z_plot',
              color='flight_id', size='size_num',size_max = 8)

fig.update_layout( 
    scene = dict(xaxis = dict(title="Easting (feet)"),
                 yaxis = dict(title="Northing (feet)"),
                 zaxis = dict(title="Vertical (feet)"),
                ),
    width=900,
    height=900,
    margin=dict(r=20, l=10, b=10, t=10),
    showlegend=False,
    xaxis = {"title":{"text":"Cat"}})
fig.show()

### Writing LAS file

In [ ]:
# Write square_points_bldg to file
inFile = File(file_dir+'10552_NYU_M2 - Scanner 1 - 190511_164039_1 - originalpoints.laz', mode='r')
# Convert DF into tuples that laspy wants
void = [(tuple(r[columns_point_cloud]),) for i,r in square_points_laefer.iterrows()]
# Export
outFile1 = File("../../Data/parking_lot/flat_parking_lot_laefer.las", mode = "w",header = inFile.header)
outFile1.points = void
outFile1.close()

In [ ]:
# NYC data
# Write square_points_bldg to file
inFile_nyc = File(nyc_file_dir+'975172.las', mode='r')
# Convert DF into tuples that laspy wants
void = [(tuple(r[columns_point_cloud]),) for i,r in square_points_nyc.iterrows()]
# Export
outFile1 = File("../../Data/parking_lot/flat_parking_lot_nyc.las", mode = "w",header = inFile.header)
outFile1.points = void
outFile1.close()

## Plotting charts from previous updates

In [ ]:
norm_vector,points,square_points,_ = plane_fit(square_points)

# Add distance from flat plane with norm (x,y,z) = (0,0,1)
square_points['dist_from_flat']=np.array([np.dot(point,np.array([0,0,1])) for point in points])

# remove data points >5 feet below plane.
outliers = square_points[square_points['dist_from_plane']<-5].index
square_points = square_points.drop(outliers)

In [ ]:
def plot_scan_angle_dist_from_plane(df,distance_metric):
    x = abs(df['scan_angle'])*.006
    y = df[distance_metric]
    plt.figure(figsize=(15,15))
    plt.plot(x,y,'xb')
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(x,p(x),"r--")
    plt.xlabel("Scan angle (degrees)")
    plt.ylabel("Point distance from plane")
    print("y={:2.8f}x+{:2.8f}".format(z[0],z[1]))
    plt.title("Scan Angle vs Distance to Fitted Plane")
plot_scan_angle_dist_from_plane(square_points,'dist_from_plane')

In [ ]:
plt.plot(range(len(square_points)),square_points['scan_angle'],'x')

In [ ]:
plot_scan_angle_dist_from_plane(square_points,'dist_from_flat')

In [ ]:
# Chart from slides showing points per run
labels = [pt[0][11:-4] for pt in pts_from_scan]
num_points = [pt[1]+.01 for pt in pts_from_scan]
plt.figure(figsize=(25,20))
plt.bar(labels,num_points,)
plt.xticks(rotation=45,fontsize=20)
plt.yticks(np.arange(0, max(num_points), step=(max(num_points)/10)),fontsize=20)
plt.ylabel("Number of points from run",fontsize=20)
plt.xlabel("Run ID",fontsize=20)